## Day 10: Performance Optimization(Combined Dataset)

In [0]:
#Step 0: Create a Combined Silver Table(Foundation)
from pyspark.sql import functions as F

#Read October Silver
df_oct = spark.table("silver_events_oct")

#Read November Silver
df_nov = spark.table("silver_df_nov_realworld")

#Align Schemas Explicitly
common_cols = [
    "event_time", "event_type", "product_id", "category_id", 
    "category_code","brand", "price", "user_id", 
    "user_session", "event_date", "price_tier"]

df_oct = df_oct.select(common_cols)
df_nov = df_nov.select(common_cols)

#Combine datasets
df_silver_all = df_oct.unionByName(df_nov)

#save combined Silver table
df_silver_all.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_events_all")

#verify
spark.table("silver_events_all").count()

109585110

In [0]:
#Step 1: Analyze Query Execution Table
spark.sql("""
SELECT *
FROM silver_events_all
WHERE event_type = "purchase"
""").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [silver_events_all], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, event_date: date, price_tier: string
Project [event_time#15771, event_type#15772, product_id#15773, category_id#15774L, category_code#15775, brand#15776, price#15777, user_id#15778, user_session#15779, event_date#15780, price_tier#15781]
+- Filter (event_type#15772 = purchase)
   +- SubqueryAlias workspace.default.silver_events_all
      +- Relation workspace.default.silver_events_all[event_time#15771,event_type#15772,product_id#15773,category_id#15774L,category_code#15775,brand#15776,price#15777,user_id#15778,user_session#15779,event_date#15780,price_tier#15781] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_type#15772) AND (event_type#15772 = p

In [0]:
%sql
-- Step 2: Create a Parttioned Silver Table
-- Now we optimize the table structure
CREATE TABLE silver_events_all_part
USING delta
PARTITIONED BY(event_date,event_type)
AS
SELECT * FROM silver_events_all

num_affected_rows,num_inserted_rows


In [0]:
#Step 3: Re-check Query plan
spark.sql("""
SELECT *
FROM silver_events_all_part
WHERE event_type = "purchase"
    """).explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [silver_events_all_part], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, event_date: date, price_tier: string
Project [event_time#20022, event_type#20023, product_id#20024, category_id#20025L, category_code#20026, brand#20027, price#20028, user_id#20029, user_session#20030, event_date#20031, price_tier#20032]
+- Filter (event_type#20023 = purchase)
   +- SubqueryAlias workspace.default.silver_events_all_part
      +- Relation workspace.default.silver_events_all_part[event_time#20022,event_type#20023,product_id#20024,category_id#20025L,category_code#20026,brand#20027,price#20028,user_id#20029,user_session#20030,event_date#20031,price_tier#20032] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_type#20023) AND (event

In [0]:
%sql
-- Step 4: OPTIMIZE + ZORDER
OPTIMIZE silver_events_all_part
ZORDER BY(user_id, product_id);

path,metrics
,"List(13, 4, List(36660025, 94727348, 4.8811531384615384E7, 13, 634549908), List(95534407, 199003854, 1.6942062375E8, 4, 677682495), 182, List(minCubeSize(107374182400), List(0, 0), List(182, 3622939385), 0, List(4, 677682495), 4, null), null, 0, 1, 182, 178, false, 0, 0, 1768691303903, 1768691326772, 8, 4, null, List(0, 0), null, 11, 11, 39109, 0, null)"


In [0]:
#Step 5: Benchmark Performance(Before vs After)
#Before (Unoptimized)

import time
start = time.time()
spark.sql("""
SELECT *
FROM silver_events_all
WHERE event_type = 'purchase'
AND event_date = '2019-11-10'
    """).count()

print(f"Unoptimized time: {time.time() - start:.2f} seconds")

#After (Partitioned + ZORDER)
start = time.time()
spark.sql("""
SELECT *
FROM silver_events_all_part
WHERE event_type = 'purchase'
AND event_date = '2019-11-10'
""").count()

print(f"Optimized time: {time.time() - start:.2f} seconds")

Unoptimized time: 0.70 seconds
Optimized time: 0.83 seconds


In [0]:
#Step 5.1: Benchmark Performance(Before vs After)
#Before (Unoptimized)

import time
start = time.time()
spark.sql("""
SELECT *
FROM silver_events_all
WHERE user_id = 518085591
    """).count()

print(f"Unoptimized time: {time.time() - start:.2f} seconds")

#After (Partitioned + ZORDER)
start = time.time()
spark.sql("""
SELECT *
FROM silver_events_all_part
WHERE user_id = 518085591
""").count()

print(f"Optimized time: {time.time() - start:.2f} seconds")

Unoptimized time: 0.65 seconds
Optimized time: 0.84 seconds


In [0]:
#Step 6: Cache for Iterative Analytics
#Cache optimized table

try:
    spark.table("silver_events_all_part").cache().count()
except Exception as e:
    print("Caching not supported on Serverless compute")


Caching not supported on Serverless compute
